In [42]:
import sys
import pandas as pd
sys.path.append('/Users/smallhamster/Documents/research/jokesclustering/') # ваш путь до корня проекта
from components import FasttextVectorizer, KmeansSimpleClusterExtractor
from vector_clustering.abstract.model import Model
from vector_clustering.data.manager import get_jokes_as_dataframe, load_pandas_csv, load, ARANEUM_FASTTEXT
from sklearn.cluster import KMeans

### 1. Инициализируем все необходимое

In [3]:
jokes = get_jokes_as_dataframe()
jokes.head()

,joke_text
0,"Мужчину трудно задеть за живое, но уж если зад..."
1,В нашем кемпинге строго запрещено людям разног...
2,А как хорошо у девушек начинается: любимый: ми...
3,"Только у нас фраза ""поправить здоровье"" означа..."
4,"Одна белка случайно попробовала пиво и поняла,..."


In [43]:
selected = load_pandas_csv('100_manual_selection.csv')
selected.head()

,joke_index,joke_text,joke_topics
0,75622,Вот и минул ещё один день дурака. Но хватит о ...,политика
1,88009,Китайские космонавты после успешно завершенной...,национальность
2,20296,"Украина - единственная страна мира, где грипп ...",политика
3,45900,Временно исполняющий обязанности президента ма...,политика
4,52912,"Решение наболевшего дела проще, чем вы там себ...",жизнь


In [44]:
topics = selected.joke_topics.values
unique_topics = set()
for topic in topics:
    unique_topics.update([x.strip() for x in topic.split(',')])

Ручной анализ тем в 100 случайных шутках, показал, что мы наблюдаем следующие темы:

In [45]:
print(unique_topics)

{'национальность', 'компьютер', 'армия', 'политика', 'менты', 'алкоголь', 'история', 'деньги', 'еда', 'животные', 'семья', 'пол', 'штирлиц', 'спорт', 'пошлятина', 'отдых', 'студенты', 'непонятно', 'работа', 'лгбт', 'преступление', 'эстрада', 'плагиат', 'праздники', 'лень', 'туристы', 'медицина', 'расизм', 'игра слов', 'погода', 'сарказм', 'отношения', 'мигранты', 'имя', 'экономика', 'мат', 'жизнь', 'зависть', 'образование', 'тюрьма', 'жадность', 'наркота'}


In [46]:
len(unique_topics)

42

предположим, что на самом деле тем на 20% больше, откуда следует, что будем искать следующее число кластеров:

In [ ]:
len(unique_topics) + int(len(unique_topics) * 0.2)

### 2. Выделим кластеры

Проинициалазируем все необходимое для работы: (если нужно)

In [ ]:
vectorizer = FasttextVectorizer()

In [ ]:
kmeans_model = KMeans(n_clusters=50, random_state=42)

In [ ]:
cluster_extractor = KmeansSimpleClusterExtractor(kmeans_model)

In [ ]:
model = Model(vectorizer, cluster_extractor)

Подготови тексты и обучим модель:

In [ ]:
texts = jokes.joke_text.values

In [ ]:
texts.shape

In [ ]:
demo = model.fit(texts)

In [ ]:
from sklearn.externals import joblib
joblib.dump(model.get_model(), 'knn_fasttext.pkl') 

In [ ]:
jokes['cluster_id'] = model.get_model().labels_

In [ ]:
def get_random_n_jokes(cluster_id, n):
    return jokes[jokes.cluster_id == cluster_id].sample(n)

In [ ]:
unique_ids = jokes.cluster_id.unique()
df = get_random_n_jokes(unique_ids[0], 5)
for cluster_id in unique_ids[1:]:
    df = df.append(get_random_n_jokes(cluster_id, 5))
df.head()

### 3. Оценим результаты

In [ ]:
df['score'] = 0
df.to_csv('scoring.csv', index=False)

In [ ]:
df = pd.read_csv('cluster_analysis.csv')

In [ ]:
print('\n\n'.join(df[df.cluster_id == 47].joke_text.values))

In [ ]:
for c in range(0, 50):
    print(c, jokes[jokes.cluster_id == c].shape[0])

0 - ирония (повезло ??!?) 2466
1 - праздники и алкоголь (немного зашумлен)  2011
2 - спорт 1685
3 - ШУМ 4611
4 - пошлость, ШУМ 1057
5 - медицина, шутки про врачей 1964
6 - деньги 1773
7 - ШУМ  3023
8 - ШУМ 5746
9 - евреи + милиция + ШУМ 2711
10 - IT 573
11 - политика + менталитет + ШУМ 3289
12 - ШУМ 2875
13 - политика + чиновники 2684
14 - ШУМ 3325
15 - ШУМ 4027
16 - алкоголь + ШУМ 4756
17 - еда + связанное с ней
18 - сортирный юмор + отношения
19 - национальности
20 - экономика
21 - деньги + IT
22 - космос + ШУМ
23 - чиновники + коррупция
24 - купля-продажа
25 - отношения + семья
26 - ШУМ
27 - дорожное движение + ШУМ
28 - ШУМ + игра слова
29 - бизнес
30 - про анекдоты + ШУМ
31 - армия + ШУМ
32 - политика + ШУМ
33 - Россия (национальность)
34 - ШУМ 2620
35 - ШУМ 3779
36 - пол (м/ж)
37 - пол (м/ж) + ШУМ
38 - образование 1925
39 - политика 2492
40 - пол 2578
41 - Россия + ШУМ
42 - ШУМ 3941
43 - ирония (повезло ??!?)
44 - ШУМ 3568
45 - за жизнь
46 - ирония + ШУМ 3643
47 - ШУМ 3791
48 - международные отношения
49 - пол (м/ж) + ШУМ

In [ ]:
get_random_cluster_id_n_jokes(0, 5)

### 4. Дальнейший анализ

найдем самый популяные слова каждого кластера

In [ ]:
from sklearn.externals import joblib
model = joblib.load('knn_fasttext.pkl')

In [ ]:
jokes['cluster_id'] = model.labels_

In [ ]:
import pymorphy2
from pymorphy2.tokenizers import simple_word_tokenize
import string
analyzer = pymorphy2.MorphAnalyzer()

In [ ]:
def lemmatize(text):
    text = [x for x in simple_word_tokenize(text) if x not in string.punctuation]
    text = [analyzer.parse(x)[0].normal_form for x in text]
    return ' '.join(text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def get_top_words(cluster_id, top_n=15):
    cluster = jokes[jokes.cluster_id == cluster_id]
    cluster['lemmatized'] = cluster.joke_text.apply(lemmatize)
    tfidf = TfidfVectorizer()
    tfidf.fit(cluster.lemmatized.values)
    indices = np.argsort(tfidf.idf_)[::-1]
    features = tfidf.get_feature_names()
    top_features = [features[i] for i in indices[:top_n]]
    return top_features

In [ ]:
get_top_words(10)

In [ ]:
import tqdm
results = []
for i in tqdm.tqdm(range(50)):
    results.append(get_top_words(i))

In [ ]:
results[3]

In [ ]:
with open('top-10-words-by-cluster.txt', 'w') as f:
    for i, res in enumerate(results):
        f.write('Cluster {}\n'.format(i))
        for word in res:
            f.write(word)
            f.write('\n')
        f.write('\n')

### 5.Поиск топ слов для каждого кластера на основе близости вектора слова к центроиде

In [4]:
from sklearn.externals import joblib
model = joblib.load('knn_fasttext.pkl')

In [5]:
import pymorphy2
import numpy as np
from pymorphy2.tokenizers import simple_word_tokenize
import string
analyzer = pymorphy2.MorphAnalyzer()

In [6]:
jokes['cluster_id'] = model.labels_

In [12]:
def nouns_in_joke(joke):
    words = simple_word_tokenize(joke)
    return [x.lower() for x in words if analyzer.parse(x)[0].tag.POS == 'NOUN']

def cluster_nouns(cluster_id):
    sub = jokes[jokes.cluster_id == cluster_id].joke_text.values
    sub = [nouns_in_joke(x) for x in sub]
    res = set()
    for x in sub:
        res.update(x)
    return np.array(list(res))

In [8]:
fasttext_wrapper = load(ARANEUM_FASTTEXT)

In [28]:
from sklearn.metrics.pairwise import euclidean_distances

def cluster_top_based_on_equclidian_distance(cluster_id, top=15):
    centroid_vector = model.cluster_centers_[cluster_id]
    nouns = cluster_nouns(cluster_id)
    
    vectores = []
    words = []
    for noun in nouns:
        try:
            vectores.append(fasttext_wrapper[noun])
            words.append(noun)
        except KeyError as e:
            pass
        
    vectores = np.array(vectores)
    nouns = np.array(words)
    
    distances = euclidean_distances(vectores, [centroid_vector])
    distances_idx = np.argsort(np.ravel(distances))
    min_dist_top = distances_idx[:top]
    print(distances[min_dist_top])
    words = nouns[min_dist_top]
    return '\n'.join(words)

In [30]:
print(cluster_top_based_on_equclidian_distance(2))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


[[0.1505993 ]
 [0.15260075]
 [0.15617454]
 [0.16016094]
 [0.16387317]
 [0.16812092]
 [0.17161429]
 [0.1716237 ]
 [0.17369813]
 [0.18063644]
 [0.18087688]
 [0.18315643]
 [0.18517503]
 [0.1874604 ]
 [0.1893295 ]]
прохорова-кейгбоксинг
футболистов-пофигистов
родоначальниками
зиновьева-морозова-счастливого-и-всех-всех-всех
страны-победителем
осьминог-предсказатель
спортсменов-коммунистов
тотализаторе
суперпидорасов
великобритании
миллионеров-капиталистов
футболист-матерщинник
черепах-репортер
наставления
медведеве-багминтон


In [24]:
for c in range(50):
    print(c)
    print(cluster_top_based_on_equclidian_distance(c))
    print()

0


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


покемонов-сетевик-патриот
веган-вейпер-скейтер-йог-диджей-блоггер-модель-тату-зожник-призмафото-ловец
современники
несправедливости
соображений
противоположности
соплеменников
ангелом-хранителем
презервативы
наставлений
привидений
справедливости
просторы
пристрастиях
недоразумения

1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


понедельником
знаменитости
понедельникам
понедельника
магнитогорска
константина
мужа-программиста
понедельников
владивостоке
понедельник-то
соседа-школьника
москва-владивосток
петрограде
бездельники
воскресеньем

2


...

растеренности
перфекционистов
политкорректности
остервенением
оно-приспосабливаться-не
решительноенаступление
сверхосторожность
мужа-программиста
подорожника
воображением
презервативом
действительности
презервативы
прокрастинации
петровичем

48


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


империалистами-колонизаторами
перестаните
соотечественникам
политкорректности
протестантов
провокаторами
вестербайтеров
сторонники
национал-предатели
бессовестностью
очернительства
осведомители
чекистами-патриотами
националистом
предатели

49


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


политкорректности
перфекционист-экзистенциалист
действительности
подозрительностью
рассудительности
последовательности
сторонниками
признательности
противоположности
противоположностей
сообразительностью
неполниценности
невнимательности
действительностью
родственниками



In [35]:
pd.DataFrame({'label' : model.labels_}).to_csv('labels.csv', index=False)

In [38]:
a = pd.read_csv('labels.csv')
a.label.unique().shape

(50,)

In [39]:
jokes.head(10)

,joke_text,cluster_id
0,"Мужчину трудно задеть за живое, но уж если зад...",18
1,В нашем кемпинге строго запрещено людям разног...,18
2,А как хорошо у девушек начинается: любимый: ми...,25
3,"Только у нас фраза ""поправить здоровье"" означа...",16
4,"Одна белка случайно попробовала пиво и поняла,...",30
5,ОБЪЯВЛЕНИЕ На время мирового финансового кризи...,29
6,"Мужчина рассматривает тест на беременность, и ...",18
7,"Таможенники в ступоре: никак не могут решить, ...",9
8,Есть какая-то двусмысленность в подписях на фо...,11
9,На театральной тусовке артисты театра Куклачев...,43


0 - ирония (повезло ??!?) 2466
1 - праздники и алкоголь (немного зашумлен)  2011
2 - спорт 1685
3 - ШУМ 4611
4 - пошлость, ШУМ 1057
5 - медицина, шутки про врачей 1964
6 - деньги 1773
7 - ШУМ  3023
8 - ШУМ 5746
9 - евреи + милиция + ШУМ 2711
10 - IT 573
11 - политика + менталитет + ШУМ 3289
12 - ШУМ 2875
13 - политика + чиновники 2684
14 - ШУМ 3325
15 - ШУМ 4027
16 - алкоголь + ШУМ 4756
17 - еда + связанное с ней
18 - сортирный юмор + отношения
19 - национальности
20 - экономика
21 - деньги + IT
22 - космос + ШУМ
23 - чиновники + коррупция
24 - купля-продажа
25 - отношения + семья
26 - ШУМ
27 - дорожное движение + ШУМ
28 - ШУМ + игра слова
29 - бизнес
30 - про анекдоты + ШУМ
31 - армия + ШУМ
32 - политика + ШУМ
33 - Россия (национальность)
34 - ШУМ 2620
35 - ШУМ 3779
36 - пол (м/ж)
37 - пол (м/ж) + ШУМ
38 - образование 1925
39 - политика 2492
40 - пол 2578
41 - Россия + ШУМ
42 - ШУМ 3941
43 - ирония (повезло ??!?)
44 - ШУМ 3568
45 - за жизнь
46 - ирония + ШУМ 3643
47 - ШУМ 3791
48 - международные отношения
49 - пол (м/ж) + ШУМ

In [50]:
idx = 45
vals = jokes[jokes.cluster_id == idx].sample(10).joke_text.values
for v in vals:
    print(v)
    print()

- И всё-таки время - субстанция материальная! - Мужики, давно сидите? - Литр!

- Для физических лиц у нас достаточно предъявить паспорт. Вы физическое лицо? - Нет, я астральное тело...

Британские учёные доказали, что кастрюля борща продлевает брак на сутки.

- Чем отличается Дума от цирка? - В Думе все представления закрытые.

В этом году Россию ждут большие перемены: племена майя - не ЦИК, могли и правильно посчитать.

Гаишник, который пять лет просидел с радаром в кустах, приобрёл способность к фотосинтезу.

подскажите что делать: Сын растёт тряпкой - собственной жизненной позиции не имеет, от мамы не отлипает, ничего не делает - даже лень голову от кровати поднять. возраст 7 дней.

Стечение обстоятельств образовало лужу, в которой барахталось несколько человек.

Супрематизм - неповторимые лексические обороты сантехника Петрова, вступившего в собачье дерьмо.

У нас в стране даже черту бедности надо переходить в строго отведенном месте.



взять фильтрованные шутки

1) TSNE для того, что есть<br>
2) иерархическая кластеризация для ШУМОВЫХ кластеров<br>
3) используем классификатор на рубрики, смотрим насколько имя кластера соответствует классу, который дал классификатор<br>
4) попробовать разные способы нормализации векторов предложений<br>
5) распределение слова Штирлиц